# Contextual bandit

콘텍스트 밴딧에서는 상태(state)개념을 도입한다. 이를 통해 어떤 밴딧을 다루고 있는지 알 수 있다. </br>
다시말해 하나의 밴딧만 다루던 문제에서 여러 밴딧을 다루며 에이전트의 목표는 단지 1개의 밴딧이 아니라 여러 개의 밴딧에 대해 최선의 액션을 학습하는 것으로 확장되었다. </br>
contextual: 맥락[전후 사정]과 관련된

In [2]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
import numpy as np

In [ ]:
class contextual_bandit():
    def __init__(self):
        self.state = 0
        # 밴딧들의 손잡이 목록을 작성(행: 밴딧, 열: 밴딧의 손잡이(액션))
        # 동일 행에서 각각의 칼럼은 밴딧의 손잡이
        self.bandits = np.array([[0.2, 0, -0.1], 
                                 [0.4, 0.0, -0.2], 
                                 [1.1, 1.2, 3.0]])
        self.num_bandits = self.bandits.shape[0] # 밴딧의 개수 설정(행)
        self.num_actions = self.bandits.shape[1] # 밴딧의 액션 개수 설정(열)
    
    def getBandit(self):
        # 각각의 에피소드에 대해 랜덤한 상태를 반환
        self.state = np.random.randint(0, self.num_bandits)
        
        return self.state
    
    def pullArm(self, action):
        # 랜덤한 수를 얻는다. 